In [1]:
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx

network = nx.DiGraph()

In [2]:
from xml.dom.minidom import parse
import xml.dom.minidom

# Open XML document using minidom parser
DOMTree = xml.dom.minidom.parse("migrations.xml")
collection = DOMTree.documentElement

nodes = collection.getElementsByTagName("node")
for node in nodes:
    key = int(node.getAttribute("id"))
    args = {}
    for data in node.getElementsByTagName("data"):
        args[data.getAttribute("key")] = data.childNodes.item(0).data
    network.add_node(key, attr=args)

In [3]:
edges = collection.getElementsByTagName("edge")
for edge in edges:
    key = int(edge.getAttribute("id"))
    source = int(edge.getAttribute("source"))
    target = int(edge.getAttribute("target"))
    #count = edge.getElementsByTagName("data")[0].childNodes.item(0).data
    network.add_edge(source, target, old_key=key)

In [4]:
network.number_of_nodes(), network.number_of_edges()

(6517, 9780)

In [4]:
network.number_of_nodes(), network.number_of_edges()

(235, 2101)

In [47]:
import numpy as np
import ForceBundleFunctional
import importlib
importlib.reload(ForceBundleFunctional)

# Build nodes
data_nodes = []
for node in network.nodes(data=True):
    data_nodes.append(ForceBundleFunctional.Point(float(node[1]['attr']['x']), float(node[1]['attr']['y']) * -1))
    #data_nodes.append((float(node[1]['attr']['x']), float(node[1]['attr']['y']) * -1))
#data_nodes = np.array(data_nodes, dtype=[('x', 'f4'), ('y', 'f4')])

# Build edges
data_edges = []
count = 0
limit = 100
for source, target in network.edges:
    #data_edges.append((source, target))
    if count >= limit:
        break
    count += 1
    data_edges.append(ForceBundleFunctional.Edge(data_nodes[source], data_nodes[target]))
#data_edges = np.array(data_edges, dtype=[('source', 'u2'), ('target', 'u2')])

In [50]:
10 ** 2

100

In [49]:
import ForceBundleFunctionalNumba
import importlib
importlib.reload(ForceBundleFunctionalNumba)
from functools import partial

# Set settings
eps = 1e-6
compatibility_threshold = 0.6

edge_as_vector = ForceBundleFunctionalNumba.edges_as_vector

edge_length = partial(ForceBundleFunctionalNumba.edge_length, eps)

angle_compatibility = partial(ForceBundleFunctionalNumba.angle_compatibility, edge_length)

#angle_compatibility(data_edges[69], data_edges[66])
scale_compatibility = partial(ForceBundleFunctionalNumba.scale_compatibility, edge_length)

position_compatibility = partial(ForceBundleFunctionalNumba.position_compatibility, edge_length)

are_compatible = partial(ForceBundleFunctionalNumba.are_compatible, angle_compatibility,
                          scale_compatibility, position_compatibility, compatibility_threshold)


data = ForceBundleFunctionalNumba.compute_compatibility_list(data_edges, are_compatible)

In [59]:
import numba

numba.typeof(np.array(data_edges))

ValueError: Unsupported array dtype: object